# Replication of Ramey and Zubairy (2018)* 


In this notebook, we will replicate the results of Ramey and Zubairy (2018)* using Python. The authors study the effects of government spending on output using historical data from the United States. They employ state-dependent local projections to estimate the government spending multiplier in both good and bad economic states.


*Ramey, V. A., & Zubairy, S. (2018). Government spending multipliers in good times and in bad: evidence from US historical data. Journal of political economy, 126(2), 850-901.


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import statsmodels.api as sm
import datetime

## Data

Ramey and Zubairy (2018) use quarterly data from 1899 to 2015. Their historical series include real GDP, the GDP deflator, government purchases, federal government receipts, population, the unemployment rate, interest rates, and defense news shocks.


The shocks series contains news about defense spending and is constructed using narrative identification. „This news series focuses on changes in government spending that are linked to political and military events since these changes are most likely to be independent of the state of the economy.“ (Ramey und Zubairy 2018, p. 857)

In [ ]:
data_description = pd.read_excel('data/RZDAT.xlsx', sheet_name='readme', skiprows=7, header=None)
data_description.columns = ['Variable', 'Description']
data_description

In [ ]:
raw = pd.read_excel('data/RZDAT.xlsx', sheet_name='rzdat')

# Extract year and quarter
quarters = raw['quarter']
years = quarters.astype(int)
quarters_num = ((quarters - years) * 4 + 1).astype(int)

# Combine to "YYYYQn" format
period_str = years.astype(str) + 'Q' + quarters_num.astype(str)
# Convert to Period (quarterly)
periods = pd.PeriodIndex(period_str, freq='Q')
# If you want a datetime (start of quarter)
dates = periods.to_timestamp()

raw.index = dates

raw.head()

You will need the following variables:
- `ngdp`: Nominal GDP
- `ngov`: Nominal government spending
- `pgdp`: GDP deflator
- `news`: News shocks
- `rgdp_pott6`: Potential GDP 
- `unemp`: Unemployment rate

Data transformations:
- Start at 1899Q1
- End at 2015Q4
- Create real variables by dividing by the GDP deflator (pgdp) for:
  - Nominal GDP (ngdp)
  - News shocks (news)
  - Nominal government spending (ngov)
- Normalize the following variables by potential GDP (rgdp_pott6):
  - Real GDP (rgdp)
  - Real government spending (rgov)
  - Real news shocks (news)


## Local Projections

The formula for the linear Local Projections is given by:
$$
x_{t+h} = \alpha_h + \beta_h \text{shock}_{t} + \gamma_h Z_{t-1} + \epsilon_{t+h}
$$

The control variables include 4 lags of the normalized variables for GDP,
government spending, and the news shock.

## State Dependent Local Projections

Now we will estimate the state-dependent local projections. The authors use a dummy variable for periods of slack (slack) and a dummy variable for periods of boom (boom). The formula for the state-dependent local projections is given by:
$$
y_t = I_{t-1} [\alpha_{a,h} + \beta_{a,h} \text{shock}_t + \gamma_{a,h} Z_{t-1}] + (1 - I_{t-1}) [\alpha_{b,h} + \beta_{b,h} \text{shock}_t + \gamma_{b,h} Z_{t-1}] + \epsilon_t
$$

where I is a dummy variable that indicates the state of the economy when the shock hits.


You will need to create additional variables for the state-dependent local projections:
- `slack`: Dummy variable for periods of slack (unemployment rate > 6.5%)
- `boom`: Dummy variable for periods of boom (unemployment rate < 6.5%)
- Interaction terms with the dummy variable.

## State-dependent Local Projections using regime proababilities

Estimate an alternative state-dependent Local Projections model, replacing the dummy variable for slack with a function that can be interpreted as the probability of being in a period of low growth, defined as $$ F(z_t) = \frac{\exp(-\gamma z_t)}{1 + \exp(-\gamma z_t)}$$
 For $z_t$, use the annual growth rate of GDP, and for $\gamma$, use the value 0.5. Then plot the estimated IRFs for GDP and government spending.